https://www.youtube.com/watch?v=kCc8FmEb1nY

In [28]:
# !curl https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt > datasets/shakespeare.txt
# !curl https://www.gutenberg.org/cache/epub/5186/pg5186.txt > datasets/kalevala-en.txt
!curl https://www.gutenberg.org/cache/epub/7000/pg7000.txt > datasets/kalevala-fi.txt

In [58]:
with open('datasets/kalevala-fi.txt') as f:
    text = f.read()[2329:]
    
print(len(text))
print(text[:1000])

610233
  Ensimmäinen runo


  Mieleni minun tekevi,
  aivoni ajattelevi
  lähteäni laulamahan,
  saa'ani sanelemahan,
  sukuvirttä suoltamahan,
  lajivirttä laulamahan.
  Sanat suussani sulavat,
  puhe'et putoelevat,
  kielelleni kerkiävät,
  hampahilleni hajoovat.
  Veli kulta, veikkoseni,
  kaunis kasvinkumppalini!
  Lähe nyt kanssa laulamahan,
  saa kera sanelemahan
  yhtehen yhyttyämme,
  kahta'alta käytyämme!
  Harvoin yhtehen yhymme,
  saamme toinen toisihimme
  näillä raukoilla rajoilla,
  poloisilla Pohjan mailla.
  Lyökämme käsi kätehen,
  sormet sormien lomahan,
  lauloaksemme hyviä,
  parahia pannaksemme,
  kuulla noien kultaisien,
  tietä mielitehtoisien,
  nuorisossa nousevassa,
  kansassa kasuavassa:
  noita saamia sanoja,
  virsiä virittämiä
  vyöltä vanhan Väinämöisen,
  alta ahjon Ilmarisen,
  päästä kalvan Kaukomielen,
  Joukahaisen jousen tiestä,
  Pohjan peltojen periltä,
  Kalevalan kankahilta.
  Niit' ennen isoni lauloi
  kirvesvartta vuollessansa;
  niitä äitini 

In [59]:
chars = sorted(set(text))

stoi = {c: i for i, c in enumerate(chars)}
itos = {i: c for i, c in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join(itos[i] for i in l)

print(chars)
print(enc := list(encode('Väinämöinen')))
print(decode(enc))

['\n', ' ', '!', '"', '$', '%', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ä', 'ä', 'ö']
[49, 80, 61, 66, 80, 65, 81, 61, 66, 57, 66]
Väinämöinen


In [60]:
import torch

data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([610233]) torch.int64
tensor([ 1,  1, 32, 66, 71, 61, 65, 65, 80, 61, 66, 57, 66,  1, 70, 73, 66, 67,
         0,  0,  0,  1,  1, 40, 61, 57, 64, 57, 66, 61,  1, 65, 61, 66, 73, 66,
         1, 72, 57, 63, 57, 74, 61, 10,  0,  1,  1, 53, 61, 74, 67, 66, 61,  1,
        53, 62, 53, 72, 72, 57, 64, 57, 74, 61,  0,  1,  1, 64, 80, 60, 72, 57,
        80, 66, 61,  1, 64, 53, 73, 64, 53, 65, 53, 60, 53, 66, 10,  0,  1,  1,
        71, 53, 53,  6, 53, 66, 61,  1, 71, 53, 66, 57, 64, 57, 65, 53, 60, 53,
        66, 10,  0,  1,  1, 71, 73, 63, 73, 74, 61, 70, 72, 72, 80,  1, 71, 73,
        67, 64, 72, 53, 65, 53, 60, 53, 66, 10,  0,  1,  1, 64, 53, 62, 61, 74,
        61, 70, 72, 72, 80,  1, 64, 53, 73, 64, 53, 65, 53, 60, 53, 66, 12,  0,
         1,  1, 46, 53, 66, 53, 72,  1, 71, 73, 73, 71, 71, 53, 66, 61,  1, 71,
        73, 64, 53, 74, 53, 72, 10,  0,  1,  1, 68, 73, 60, 57,  6, 57, 72,  1,
        68, 73, 72, 67, 57, 64, 57, 74, 53, 72, 10,  0,  1,  1, 63, 61, 57, 64,
       

In [61]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [62]:
block_size = 8

train_data[:block_size+1]

tensor([ 1,  1, 32, 66, 71, 61, 65, 65, 80])

In [63]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'{context} -> {target}')

tensor([1]) -> 1
tensor([1, 1]) -> 32
tensor([ 1,  1, 32]) -> 66
tensor([ 1,  1, 32, 66]) -> 71
tensor([ 1,  1, 32, 66, 71]) -> 61
tensor([ 1,  1, 32, 66, 71, 61]) -> 65
tensor([ 1,  1, 32, 66, 71, 61, 65]) -> 65
tensor([ 1,  1, 32, 66, 71, 61, 65, 65]) -> 80


In [69]:
torch.manual_seed(123)

batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print(f'''
{xb.shape=}
{xb=}
 
{yb.shape=}
{yb=}
''')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'{context.tolist()} -> {target}')


xb.shape=torch.Size([4, 8])
xb=tensor([[ 0,  1,  1, 57, 61, 63, 80, 68],
        [ 1, 65, 53, 72, 63, 53, 64, 64],
        [ 0,  1,  1, 68, 53, 64, 53, 71],
        [ 1, 71, 57, 68, 68, 67,  1, 36]])
 
yb.shape=torch.Size([4, 8])
yb=tensor([[ 1,  1, 57, 61, 63, 80, 68, 80],
        [65, 53, 72, 63, 53, 64, 64, 53],
        [ 1,  1, 68, 53, 64, 53, 71, 57],
        [71, 57, 68, 68, 67,  1, 36, 64]])

[0] -> 1
[0, 1] -> 1
[0, 1, 1] -> 57
[0, 1, 1, 57] -> 61
[0, 1, 1, 57, 61] -> 63
[0, 1, 1, 57, 61, 63] -> 80
[0, 1, 1, 57, 61, 63, 80] -> 68
[0, 1, 1, 57, 61, 63, 80, 68] -> 80
[1] -> 65
[1, 65] -> 53
[1, 65, 53] -> 72
[1, 65, 53, 72] -> 63
[1, 65, 53, 72, 63] -> 53
[1, 65, 53, 72, 63, 53] -> 64
[1, 65, 53, 72, 63, 53, 64] -> 64
[1, 65, 53, 72, 63, 53, 64, 64] -> 53
[0] -> 1
[0, 1] -> 1
[0, 1, 1] -> 68
[0, 1, 1, 68] -> 53
[0, 1, 1, 68, 53] -> 64
[0, 1, 1, 68, 53, 64] -> 53
[0, 1, 1, 68, 53, 64, 53] -> 71
[0, 1, 1, 68, 53, 64, 53, 71] -> 57
[1] -> 71
[1, 71] -> 57
[1, 71, 57] -> 68
[1, 71, 

In [70]:
xb

tensor([[ 0,  1,  1, 57, 61, 63, 80, 68],
        [ 1, 65, 53, 72, 63, 53, 64, 64],
        [ 0,  1,  1, 68, 53, 64, 53, 71],
        [ 1, 71, 57, 68, 68, 67,  1, 36]])

In [75]:
import torch
from torch import nn
from torch.nn import functional as F
torch.manual_seed(123)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx)
        return logits

m = BigramLanguageModel(vocab_size=len(chars))
out = m(xb, yb)

out.shape

torch.Size([4, 8, 82])

In [76]:
m.token_embedding_table

Embedding(82, 82)

In [77]:
out

tensor([[[ 0.3374, -0.1778, -0.3035,  ..., -1.4465,  0.2553, -0.5496],
         [ 1.0042,  0.8272, -0.3948,  ...,  2.0024, -0.5469,  1.6014],
         [ 1.0042,  0.8272, -0.3948,  ...,  2.0024, -0.5469,  1.6014],
         ...,
         [ 0.1219, -1.2203,  0.9311,  ...,  1.0900,  0.3342,  0.4057],
         [ 0.3122, -1.2654, -0.4209,  ...,  1.4228, -0.1422, -1.4169],
         [ 1.7749,  0.4905, -0.5847,  ..., -0.1341,  0.8229,  0.0723]],

        [[ 1.0042,  0.8272, -0.3948,  ...,  2.0024, -0.5469,  1.6014],
         [-0.5910,  0.2357, -1.3192,  ...,  0.5429,  0.9102,  0.1155],
         [ 1.0788, -1.3775,  0.0337,  ..., -0.9473,  2.3808,  0.2818],
         ...,
         [ 1.0788, -1.3775,  0.0337,  ..., -0.9473,  2.3808,  0.2818],
         [ 0.7726, -1.5532,  0.0867,  ...,  1.4542,  0.6126, -0.5760],
         [ 0.7726, -1.5532,  0.0867,  ...,  1.4542,  0.6126, -0.5760]],

        [[ 0.3374, -0.1778, -0.3035,  ..., -1.4465,  0.2553, -0.5496],
         [ 1.0042,  0.8272, -0.3948,  ...,  2